In [ ]:
# MULTI-USER AI IMAGE LAB

# -------- INSTALL ----------
!pip install diffusers transformers accelerate safetensors pillow torch gradio -q

In [ ]:
# -------- IMPORTS ----------
import torch, os, uuid
import gradio as gr
from PIL import Image, ImageEnhance
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionImg2ImgPipeline,
    EulerAncestralDiscreteScheduler
)

#  STEP 3: Hugging Face Token (Required for Diffusers)
HUGGING_FACE_TOKEN = ""  # Replace if expired
HUGGING_FACE_TOKEN = ""  # Replace if expired

In [ ]:
# -------- SYSTEM CONFIG ----------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
IMG_SIZE = 768
UPSCALE_SIZE = 2048

print("Device:", DEVICE)

# -------- MODEL INITIALIZATION ----------
def load_scheduler():
    return EulerAncestralDiscreteScheduler.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        subfolder="scheduler"
    )

def load_pipelines(scheduler):
    txt_pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        scheduler=scheduler,
        torch_dtype=DTYPE
    ).to(DEVICE)

    img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        scheduler=scheduler,
        torch_dtype=DTYPE
    ).to(DEVICE)

    txt_pipe.enable_attention_slicing()
    img_pipe.enable_attention_slicing()

    return txt_pipe, img_pipe

scheduler = load_scheduler()
txt2img, img2img = load_pipelines(scheduler)

# -------- QUALITY FILTER ----------
NEGATIVE_FILTER = (
    "blurry, low quality, distorted anatomy, bad hands, "
    "watermark, text, logo, oversharpen, noisy background"
)

# -------- IMAGE PIPELINE ----------
def enhance_image(img):
    img = ImageEnhance.Contrast(img).enhance(1.4)
    img = ImageEnhance.Sharpness(img).enhance(2.1)
    img = ImageEnhance.Brightness(img).enhance(1.1)
    return img

def generate_ai_image(user_prompt):
    with torch.autocast(DEVICE) if DEVICE == "cuda" else torch.no_grad():
        base_img = txt2img(
            prompt=user_prompt,
            negative_prompt=NEGATIVE_FILTER,
            guidance_scale=13,
            num_inference_steps=60,
            height=IMG_SIZE,
            width=IMG_SIZE
        ).images[0]

    refined_img = img2img(
        prompt=user_prompt,
        negative_prompt=NEGATIVE_FILTER,
        image=base_img.convert("RGB"),
        strength=0.65,
        guidance_scale=15,
        num_inference_steps=80
    ).images[0]

    refined_img = enhance_image(refined_img)
    final_img = refined_img.resize((UPSCALE_SIZE, UPSCALE_SIZE), Image.LANCZOS)

    os.makedirs("outputs", exist_ok=True)
    file_path = f"outputs/final_{uuid.uuid4().hex[:8]}.png"
    final_img.save(file_path)

    return final_img, file_path


In [ ]:
# -------- GRADIO INTERFACE ----------
with gr.Blocks() as app:
    gr.Markdown(" Ultra-HD AI Image Generator")
    gr.Markdown("High quality Stable Diffusion based image creation system")

    prompt_box = gr.Textbox(
        label="Describe your image",
        placeholder="Example: futuristic city at sunrise, ultra realistic",
        lines=2
    )

    run_btn = gr.Button("Create Image")

    output_img = gr.Image(label="Generated Image")
    download = gr.File(label="Download")

    run_btn.click(
        fn=generate_ai_image,
        inputs=prompt_box,
        outputs=[output_img, download]
    )

app.launch(share=True, debug=True)